# Bloomberg 数据导入工具

将 `data/SHSZ300_data.xlsx` 中的沪深300数据导入 DataRepository。

**数据来源**：Bloomberg（BBG）导出的 Excel  
**数据内容**：300只股票，约243个交易日  
**字段映射**：
- `PX_OPEN` → `open`
- `PX_HIGH` → `high`
- `PX_LOW`  → `low`
- `PX_LAST` → `close`
- `VOLUME`  → `volume`
- `PE_RATIO` → 存入 `market_value` 表（`pe` 字段）

**注意**：
- 本 notebook 直接调用 `DataRepository.save_ohlcv_data` / `save_market_value_data`，**绕过 DataUpdater**
- 这是合理的：Bloomberg 数据已经是干净的外部权威数据源，不需要走 AkShare 下载流程
- 如果 repository 中已有对应数据，会被**覆盖**（save_* 均为纯覆盖写入）

## Cell 1 — 环境初始化

In [1]:
import sys
import os

# 确保项目根目录在 path 中
project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

from data_process.repository import DataRepository

print('✅ 环境初始化完成')

C:\Users\mengxinchen\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


✅ 环境初始化完成


## Cell 2 — 配置参数

In [2]:
# =====================================================================
# 配置区：按需修改
# =====================================================================

# Excel 文件路径
EXCEL_PATH = './data/SHSZ300_data.xlsx'

# Repository 存储根目录（与 data_download.ipynb 保持一致）
REPO_DIR = './rawdata/repository'

# 是否导入 PE（market_value 表）
IMPORT_PE = True

# 出错时是否继续处理后续股票
SKIP_ON_ERROR = True

# =====================================================================

excel_path = Path(EXCEL_PATH)
assert excel_path.exists(), f'❌ 文件不存在: {excel_path.absolute()}'

repo = DataRepository(base_dir=REPO_DIR)

print(f'📂 Excel 文件 : {excel_path.absolute()}')
print(f'📂 Repository : {Path(REPO_DIR).absolute()}')
print(f'📊 导入 PE    : {IMPORT_PE}')

📂 Excel 文件 : d:\2026_claude\StockFrame\data\SHSZ300_data.xlsx
📂 Repository : d:\2026_claude\StockFrame\rawdata\repository
📊 导入 PE    : True


## Cell 3 — 预览数据结构

In [3]:
# 加载所有 sheet 名（即股票代码列表）
xl = pd.ExcelFile(EXCEL_PATH)
all_symbols = xl.sheet_names

print(f'共 {len(all_symbols)} 只股票')
print(f'前10个: {all_symbols[:10]}')

# 展示第一个 sheet 的原始格式
sample_raw = xl.parse(all_symbols[0])
print(f'\n--- 原始列名（以 {all_symbols[0]} 为例）---')
for col in sample_raw.columns:
    print(f'  {col!r}')
print(f'\n日期范围: {sample_raw["Date"].min().date()} ~ {sample_raw["Date"].max().date()}')
print(f'共 {len(sample_raw)} 行')

共 300 只股票
前10个: ['000001', '000002', '000063', '000100', '000157', '000166', '000301', '000333', '000338', '000408']

--- 原始列名（以 000001 为例）---
  'Date'
  "('000001 CH EQUITY', 'PX_OPEN')"
  "('000001 CH EQUITY', 'PX_HIGH')"
  "('000001 CH EQUITY', 'PX_LOW')"
  "('000001 CH EQUITY', 'PX_LAST')"
  "('000001 CH EQUITY', 'VOLUME')"
  "('000001 CH EQUITY', 'PE_RATIO')"

日期范围: 2025-02-26 ~ 2026-02-26
共 243 行


## Cell 4 — 列名解析函数

In [4]:
import re

# Bloomberg 列名格式："('000001 CH EQUITY', 'PX_OPEN')" 或直接 'PX_OPEN'
# 统一提取字段名部分
BBG_FIELD_MAP = {
    'PX_OPEN'  : 'open',
    'PX_HIGH'  : 'high',
    'PX_LOW'   : 'low',
    'PX_LAST'  : 'close',
    'VOLUME'   : 'volume',
    'PE_RATIO' : 'pe',
}

def parse_bbg_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    将 Bloomberg 导出的元组格式列名统一重命名。
    支持两种格式：
      - ("000001 CH EQUITY", "PX_OPEN")  → 提取第二个元素
      - "PX_OPEN"                          → 直接映射
      - "Date"                             → 保留为 date
    """
    new_cols = {}
    for col in df.columns:
        col_str = str(col)
        if col_str == 'Date':
            new_cols[col] = 'date'
            continue
        # 从元组字符串中提取字段名：("xxx", "PX_OPEN") → PX_OPEN
        m = re.search(r"'([A-Z_]+)'\s*\)", col_str)
        field = m.group(1) if m else col_str
        new_cols[col] = BBG_FIELD_MAP.get(field, field.lower())
    return df.rename(columns=new_cols)

# 验证
sample = parse_bbg_columns(sample_raw)
print('重命名后列名:', list(sample.columns))
print(sample.head(3).to_string())

重命名后列名: ['date', 'open', 'high', 'low', 'close', 'volume', 'pe']
        date   open   high    low  close    volume      pe
0 2025-02-26  11.47  11.60  11.47  11.52  84164552  4.9196
1 2025-02-27  11.53  11.63  11.46  11.62  97730975  4.9623
2 2025-02-28  11.58  11.68  11.50  11.53  94908806  4.9239


## Cell 5 — 单股数据转换函数

In [5]:
def build_ohlcv(df: pd.DataFrame, symbol: str) -> pd.DataFrame:
    """
    构建符合 DataRepository.save_ohlcv_data 要求的 OHLCV DataFrame。

    要求：
    - 索引为 DatetimeIndex（名称为 'date'）
    - 必须包含 open / high / low / close / volume
    - 可选列：turnover / amplitude 等
    """
    ohlcv_cols = ['open', 'high', 'low', 'close', 'volume']
    result = df[ohlcv_cols].copy()

    # 设置日期索引
    result.index = pd.to_datetime(df['date'])
    result.index.name = 'date'
    result.sort_index(inplace=True)

    # 数值类型
    for col in ohlcv_cols:
        result[col] = pd.to_numeric(result[col], errors='coerce')

    # 添加 symbol 列（DataValidator 会检查）
    result['symbol'] = symbol

    # 删除全空行
    result.dropna(subset=['close'], inplace=True)
    return result


def build_market_value(df: pd.DataFrame, symbol: str) -> pd.DataFrame:
    """
    构建符合 DataRepository.save_market_value_data 要求的市值 DataFrame。

    Excel 中只有 PE_RATIO，total/circulating market cap 用 NaN 填充。
    """
    result = pd.DataFrame(index=pd.to_datetime(df['date']))
    result.index.name = 'date'
    result['total_market_cap'] = np.nan          # Excel 无此字段
    result['circulating_market_cap'] = np.nan    # Excel 无此字段
    result['pe'] = pd.to_numeric(df['pe'].values, errors='coerce') if 'pe' in df.columns else np.nan
    result.sort_index(inplace=True)
    result.dropna(subset=['pe'], inplace=True)
    return result


# 快速验证
test_df = parse_bbg_columns(xl.parse(all_symbols[0]))
test_ohlcv = build_ohlcv(test_df, all_symbols[0])
test_mv    = build_market_value(test_df, all_symbols[0])
print('OHLCV shape:', test_ohlcv.shape, '  columns:', list(test_ohlcv.columns))
print(test_ohlcv.head(2).to_string())
print('\nMarketValue shape:', test_mv.shape, '  columns:', list(test_mv.columns))
print(test_mv.head(2).to_string())

OHLCV shape: (243, 6)   columns: ['open', 'high', 'low', 'close', 'volume', 'symbol']
             open   high    low  close    volume  symbol
date                                                    
2025-02-26  11.47  11.60  11.47  11.52  84164552  000001
2025-02-27  11.53  11.63  11.46  11.62  97730975  000001

MarketValue shape: (243, 3)   columns: ['total_market_cap', 'circulating_market_cap', 'pe']
            total_market_cap  circulating_market_cap      pe
date                                                        
2025-02-26               NaN                     NaN  4.9196
2025-02-27               NaN                     NaN  4.9623


## Cell 6 — 批量导入

In [6]:
results = []   # (symbol, status, ohlcv_rows, mv_rows, error)

for symbol in tqdm(all_symbols, desc='导入进度'):
    try:
        # 1. 读取并重命名列
        raw = xl.parse(symbol)
        df  = parse_bbg_columns(raw)

        # 2. 导入 OHLCV
        ohlcv = build_ohlcv(df, symbol)
        repo.save_ohlcv_data(symbol, ohlcv)
        ohlcv_rows = len(ohlcv)

        # 3. 导入 PE（market_value）
        mv_rows = 0
        if IMPORT_PE and 'pe' in df.columns:
            mv = build_market_value(df, symbol)
            if not mv.empty:
                repo.save_market_value_data(symbol, mv)
                mv_rows = len(mv)

        results.append((symbol, 'ok', ohlcv_rows, mv_rows, None))

    except Exception as e:
        results.append((symbol, 'error', 0, 0, str(e)))
        if not SKIP_ON_ERROR:
            raise

print('\n✅ 导入完成')

导入进度:   0%|          | 0/300 [00:00<?, ?it/s]


✅ 导入完成


## Cell 7 — 导入结果统计

In [7]:
result_df = pd.DataFrame(results, columns=['symbol', 'status', 'ohlcv_rows', 'mv_rows', 'error'])

ok_df  = result_df[result_df['status'] == 'ok']
err_df = result_df[result_df['status'] == 'error']

print('=' * 60)
print(f'总计       : {len(result_df)} 只股票')
print(f'✅ 成功     : {len(ok_df)} 只')
print(f'❌ 失败     : {len(err_df)} 只')
print(f'OHLCV 总行 : {ok_df["ohlcv_rows"].sum():,}')
if IMPORT_PE:
    print(f'PE 总行    : {ok_df["mv_rows"].sum():,}')
print('=' * 60)

if not err_df.empty:
    print('\n❌ 失败明细：')
    for _, row in err_df.iterrows():
        print(f'  {row["symbol"]}: {row["error"]}')

总计       : 300 只股票
✅ 成功     : 286 只
❌ 失败     : 14 只
OHLCV 总行 : 69,498
PE 总行    : 63,862

❌ 失败明细：
  002049: 数据验证失败: 存在空值: {'volume': 10}
  300442: 数据验证失败: 存在空值: {'volume': 6}
  301269: 数据验证失败: 存在空值: {'volume': 10}
  600150: 数据验证失败: 存在空值: {'volume': 4}
  600438: 数据验证失败: 存在空值: {'volume': 2}
  600803: 数据验证失败: 存在空值: {'volume': 6}
  601059: 数据验证失败: 存在空值: {'volume': 20}
  601088: 数据验证失败: 存在空值: {'volume': 10}
  601995: 数据验证失败: 存在空值: {'volume': 20}
  603019: 数据验证失败: 存在空值: {'volume': 10}
  688012: 数据验证失败: 存在空值: {'volume': 9}
  688041: 数据验证失败: 存在空值: {'volume': 10}
  688126: 数据验证失败: 存在空值: {'volume': 8}
  688981: 数据验证失败: 存在空值: {'volume': 6}


## Cell 8 — 验证：读取几只股票的数据

In [8]:
# 随机抽取几只验证
verify_symbols = ['000001', '600519', '300750']
verify_symbols = [s for s in verify_symbols if s in all_symbols]

for sym in verify_symbols:
    try:
        ohlcv = repo.load_ohlcv_data(sym)
        print(f'\n✅ {sym} OHLCV: {len(ohlcv)} 行，{ohlcv.index.min().date()} ~ {ohlcv.index.max().date()}')
        print(ohlcv.tail(2).to_string())

        if IMPORT_PE:
            mv = repo.load_market_value_data(sym)
            if not mv.empty:
                print(f'   PE: {len(mv)} 行，最新 PE = {mv["pe"].iloc[-1]:.2f}')
    except Exception as e:
        print(f'❌ {sym}: {e}')


✅ 000001 OHLCV: 243 行，2025-02-26 ~ 2026-02-26
             open   high    low  close     volume  symbol
date                                                     
2026-02-25  10.91  10.95  10.78  10.86  106313487       1
2026-02-26  10.86  10.91  10.80  10.87   71273019       1
   PE: 243 行，最新 PE = 4.78

✅ 600519 OHLCV: 243 行，2025-02-26 ~ 2026-02-26
              open     high      low    close   volume  symbol
date                                                          
2026-02-25  1470.0  1509.66  1467.88  1491.66  3688067  600519
2026-02-26  1486.6  1489.49  1464.67  1466.21  3295801  600519
   PE: 243 行，最新 PE = 20.44

✅ 300750 OHLCV: 243 行，2025-02-26 ~ 2026-02-26
              open   high    low   close    volume  symbol
date                                                      
2026-02-25  363.69  366.6  360.1  362.18  18716639  300750
2026-02-26  356.05  358.2  343.5  346.00  43541381  300750
   PE: 243 行，最新 PE = 24.12


## Cell 9 — 查看 Repository 库存

In [9]:
ohlcv_symbols = repo.list_symbols('ohlcv')
mv_symbols    = repo.list_symbols('market_value')

print(f'Repository 中 OHLCV   : {len(ohlcv_symbols)} 只')
print(f'Repository 中 市值/PE : {len(mv_symbols)} 只')
print(f'\n前10只: {sorted(ohlcv_symbols)[:10]}')

Repository 中 OHLCV   : 286 只
Repository 中 市值/PE : 270 只

前10只: ['000001', '000002', '000063', '000100', '000157', '000166', '000301', '000333', '000338', '000408']


## Cell 10 — 申万行业分类导入

从 AkShare 获取申万三级行业分类，将沪深300成分股的行业归属保存到 Repository。

**流程**：
1. 获取申万一/二/三级行业列表，构建层级映射
2. 遍历258个三级行业，逐一拉取成分股（每次间隔0.3s）
3. 过滤出本次 Bloomberg 导入的300只股票
4. 保存到 `reference/industry_classification.csv`

**注意**：全量拉取约需1~2分钟，请耐心等待。

In [10]:
import time
import akshare as ak

# ---------------------------------------------------------------
# 第一步：构建 三级行业名 → 一/二级行业名 的映射表
# sw3.上级行业 = 二级行业名，sw2.上级行业 = 一级行业名
# ---------------------------------------------------------------
print('正在获取申万行业层级信息...')
sw1_info = ak.sw_index_first_info()
sw2_info = ak.sw_index_second_info()
sw3_info = ak.sw_index_third_info()

# sw2名称 → sw1名称
sw2_to_sw1 = dict(zip(sw2_info['行业名称'], sw2_info['上级行业']))
# sw3名称 → sw2名称（即 sw3.上级行业）
sw3_to_sw2 = dict(zip(sw3_info['行业名称'], sw3_info['上级行业']))

print(f'一级行业: {len(sw1_info)} 个，二级行业: {len(sw2_info)} 个，三级行业: {len(sw3_info)} 个')

# ---------------------------------------------------------------
# 第二步：遍历全部三级行业，拉取成分股
# ---------------------------------------------------------------
print('\n正在拉取各行业成分股（约258个三级行业）...')
rows = []
failed_industries = []

for _, ind_row in tqdm(sw3_info.iterrows(), total=len(sw3_info), desc='拉取行业成分股'):
    sw3_code = ind_row['行业代码']
    sw3_name = ind_row['行业名称']
    sw2_name = sw3_to_sw2.get(sw3_name, '')
    sw1_name = sw2_to_sw1.get(sw2_name, '')

    try:
        time.sleep(0.3)
        cons = ak.sw_index_third_cons(symbol=sw3_code)
        if cons is None or cons.empty:
            continue
        for _, stock_row in cons.iterrows():
            raw_code = str(stock_row['股票代码'])
            symbol = raw_code.split('.')[0].zfill(6)   # 600519.SH → 600519
            rows.append({
                'symbol'   : symbol,
                'name'     : str(stock_row.get('股票简称', '')),
                'sw1_name' : sw1_name,
                'sw2_name' : sw2_name,
                'sw3_name' : sw3_name,
                'sw3_code' : sw3_code,
            })
    except Exception as e:
        failed_industries.append((sw3_code, sw3_name, str(e)))

industry_df = (
    pd.DataFrame(rows)
    .drop_duplicates(subset='symbol', keep='first')
    .reset_index(drop=True)
)

print(f'\n✅ 获取完成：{len(industry_df)} 只股票有行业信息')
if failed_industries:
    print(f'⚠️  {len(failed_industries)} 个行业获取失败：')
    for code, name, err in failed_industries[:5]:
        print(f'   {code} {name}: {err}')

industry_df.head(5)


正在获取申万行业层级信息...
一级行业: 31 个，二级行业: 124 个，三级行业: 258 个

正在拉取各行业成分股（约258个三级行业）...


拉取行业成分股:   0%|          | 0/258 [00:00<?, ?it/s]


✅ 获取完成：4917 只股票有行业信息


,symbol,name,sw1_name,sw2_name,sw3_name,sw3_code
0,600313,农发种业,农林牧渔,种植业,种子,850111.SI
1,000713,国投丰乐,农林牧渔,种植业,种子,850111.SI
2,000998,隆平高科,农林牧渔,种植业,种子,850111.SI
3,300087,荃银高科,农林牧渔,种植业,种子,850111.SI
4,300189,神农种业,农林牧渔,种植业,种子,850111.SI


In [11]:
# ---------------------------------------------------------------
# 第三步：过滤出本次导入的300只股票，并保存到 Repository
# ---------------------------------------------------------------
bbg_set = set(all_symbols)   # Excel 中的300只股票代码
industry_bbg = industry_df[industry_df['symbol'].isin(bbg_set)].copy()

covered     = set(industry_bbg['symbol'])
not_covered = sorted(bbg_set - covered)

print('=' * 60)
print(f'Bloomberg 300只中，有申万行业信息 : {len(covered)} 只')
print(f'无申万行业信息                   : {len(not_covered)} 只')
if not_covered:
    print(f'  缺少行业的股票: {not_covered}')
print('=' * 60)

# 保存
path = repo.save_industry_classification(industry_bbg)
print(f'\n✅ 行业分类数据已保存至: {path}')
print(f'   行数: {len(industry_bbg)}')

Bloomberg 300只中，有申万行业信息 : 269 只
无申万行业信息                   : 31 只
  缺少行业的股票: ['000408', '000708', '000792', '002027', '002129', '002311', '002460', '002466', '002594', '003816', '300274', '300413', '300896', '300979', '600009', '600050', '600111', '600438', '600875', '600938', '600941', '601127', '601728', '601888', '601985', '603799', '605117', '688169', '688303', '688396', '688981']

✅ 行业分类数据已保存至: rawdata\repository\reference\industry_classification.csv
   行数: 269


In [12]:
# ---------------------------------------------------------------
# 第四步：验证读取 + 行业分布统计
# ---------------------------------------------------------------
ind_loaded = repo.load_industry_classification()

print(f'✅ 读取行业分类数据：{len(ind_loaded)} 行')
print(f'   列：{list(ind_loaded.columns)}\n')

# 一级行业分布
sw1_dist = (
    ind_loaded.groupby('sw1_name')['symbol']
    .count()
    .sort_values(ascending=False)
    .rename('股票数')
    .reset_index()
    .rename(columns={'sw1_name': '一级行业'})
)
print('── 一级行业分布 ──────────────────────────')
print(sw1_dist.to_string(index=False))

# 样例
print('\n── 样例数据（前5行）─────────────────────')
print(ind_loaded[['symbol', 'name', 'sw1_name', 'sw2_name', 'sw3_name']].head(5).to_string(index=False))

✅ 读取行业分类数据：269 行
   列：['symbol', 'name', 'sw1_name', 'sw2_name', 'sw3_name', 'sw3_code']

── 一级行业分布 ──────────────────────────
一级行业  股票数
  电子   29
非银金融   27
  银行   24
医药生物   23
电力设备   17
交通运输   16
 计算机   16
  汽车   12
食品饮料   12
公用事业   11
有色金属   10
建筑装饰    9
国防军工    8
基础化工    8
机械设备    8
  煤炭    6
石油石化    6
  通信    6
家用电器    4
农林牧渔    4
 房地产    4
建筑材料    3
轻工制造    2
  钢铁    2
  传媒    1
商贸零售    1

── 样例数据（前5行）─────────────────────
symbol name sw1_name sw2_name sw3_name
300999  金龙鱼     农林牧渔    农产品加工     粮油加工
000876  新希望     农林牧渔      养殖业     生猪养殖
002714 牧原股份     农林牧渔      养殖业     生猪养殖
300498 温氏股份     农林牧渔      养殖业     生猪养殖
002648 卫星化学     基础化工     化学原料   其他化学原料
